# Imports

In [1]:
import requests
import json
import pandas as pd
import psycopg2
from db_manipulation import Postgres_exec

# JSON

In [ ]:
# Getting response from Mercado Bitcoin's API
response_API = requests.get('https://www.mercadobitcoin.net/api/BTC/day-summary/2022/7/5/')

In [ ]:
# Adding the response to a variable
data = response_API.text

In [ ]:
data

In [ ]:
type(data)

In [ ]:
# Converting reponse to JSON standard
data_json = json.loads(data)

In [ ]:
type(data_json)

In [ ]:
# Printing in JSON format
data_json

In [ ]:
# creating Pandas dataframe from the JSON using "json_normalize" method
df = pd.json_normalize(data_json)

In [ ]:
# Printing dataframe
df.head()

# Database

In [ ]:
# Creating connection to Postgres database on local machine
conn = psycopg2.connect(
    database='projeto-pos', user='postgres',
    password='123456', host='localhost', port='5433'
)

In [ ]:
#Setting auto commit true
conn.autocommit = True

In [ ]:
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

In [ ]:
#Retrieving data
cursor.execute('''SELECT * from projeto.bitcoin_history''')

In [ ]:
#Fetching 1st row from the table
result = cursor.fetchone();
print(result)

In [ ]:
postgres_insert_query = """ INSERT INTO projeto.bitcoin_history ("date_summary",
                                                                "opening",
                                                                "closing",
                                                                "lowest",
                                                                "highest", 
                                                                "volume",
                                                                "quantity", 
                                                                "amount",
                                                                "avg_price") 
                                                                VALUES 
                                                                (%s ,%s ,%s ,%s ,%s ,%s ,%s ,%s ,%s)"""

In [ ]:
# Creating a sample record to input
record_to_insert = ('2000-05-07', 123456.12345678, 123456.12345678, 123456.12345678, 123456.12345678,
                    12345678.12345678, 123.12345678, 10000, 123456.12345678)

In [ ]:
cursor = Postgres_exec.insert_into(values=record_to_insert)

In [ ]:
# Executing the query. First argument is the SQL itself and the second argument is the data to be inputted.
cursor.execute(postgres_insert_query, record_to_insert)

In [ ]:
# Commiting the query
conn.commit()

In [ ]:
#Retrieving data
cursor.execute('''SELECT * from projeto.bitcoin_history''')

In [ ]:
# Assigning the SELECT to result variable
result = cursor.fetchall()

In [ ]:
# Printing result. As we can see, our query has run successfully.
print(result)

# Tests DB functions

In [2]:
db = Postgres_exec()

In [4]:
# Creating a sample record to input
record_to_insert2 = ('2003-05-07', 123456.12345678, 123456.12345678, 123456.12345678, 123456.12345678,
                    12345678.12345678, 123.12345678, 10000, 123456.12345678)

In [5]:
db.insert_into(values=record_to_insert2)

In [6]:
db.select_all()

+----------------+-----------+-----------+----------+-----------+-------------+------------+----------+-------------+
| date_summary   |   opening |   closing |   lowest |   highest |      volume |   quantity |   amount |   avg_price |
|----------------+-----------+-----------+----------+-----------+-------------+------------+----------+-------------|
| 1999-05-07     |    123456 |    123456 |   123456 |    123456 | 1.23457e+07 |    123.123 |    10000 |      123456 |
| 2000-05-07     |    123456 |    123456 |   123456 |    123456 | 1.23457e+07 |    123.123 |    10000 |      123456 |
| 2001-05-07     |    123456 |    123456 |   123456 |    123456 | 1.23457e+07 |    123.123 |    10000 |      123456 |
| 2002-05-07     |    123456 |    123456 |   123456 |    123456 | 1.23457e+07 |    123.123 |    10000 |      123456 |
| 2003-05-07     |    123456 |    123456 |   123456 |    123456 | 1.23457e+07 |    123.123 |    10000 |      123456 |
+----------------+-----------+-----------+----------+---